# Phenotype Synergy Analysis

This notebook contains code to interprete results from the synergy score analysis. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import math
import os
import sys
import logging
mf_module_path = os.path.abspath(os.path.join('../python'))
if mf_module_path not in sys.path:
    sys.path.append(mf_module_path)
import mf
import mf_random
import hpoutil
import networkx
import obonet
import pickle

In [2]:
hpo = hpoutil.HPO('/Users/zhangx/git/human-phenotype-ontology/hp.obo')

## Synergy among Lab-derived Abnormal Phenotypes

In [ ]:
with open('synergies.obj', 'rb') as synergies_file:
    deserialized = pickle.load(synergies_file)

In [ ]:
len(deserialized)

Plot the synergy scores. They are all pretty small.

In [ ]:
heart_failure = deserialized['428']
plt.hist(heart_failure.pairwise_synergy().flat, bins=10)
plt.show()

In [ ]:
# load p values estimated from empirical distributions
path = '/Users/zhangx/git/MIMIC_HPO/src/main/resources/p_value_map_428.obj'
with open(path, 'rb') as f:
    p_value_map = pickle.load(f)
p_heart_failure = p_value_map['428']

In [ ]:
data = heart_failure.pairwise_synergy_labeled_with_p_values(p_heart_failure)
data.reset_index(drop=True).head()

remove duplication record: (HP1, HP2) is the same to (HP2, HP1)

In [ ]:
mask = np.array([hpo.has_dependency(data.P1[i], data.P2[i]) for i in np.arange(data.shape[0])])
S_heart_failure = data.loc[np.logical_not(mask), :].sort_values(by = 'synergy', ascending=False)
S_heart_failure = S_heart_failure.loc[S_heart_failure.P1 < S_heart_failure.P2, :]
S_heart_failure['P1_label'] = np.array([hpo.term_id2name_map().get(termid) for termid in S_heart_failure.P1])
S_heart_failure['P2_label'] = np.array([hpo.term_id2name_map().get(termid) for termid in S_heart_failure.P2])
top5percent_synergy_pair_heart_failure = S_heart_failure.iloc[0:math.ceil(0.05 * len(S_heart_failure)), :]
top5percent_synergy_pair_heart_failure.head(2)

In [ ]:
top5percent_synergy_pair_heart_failure.to_csv('top5percent_synergy_pair_heart_failure.csv', index=False)

In [ ]:
acute_renal_failure = deserialized['584']
plt.hist(acute_renal_failure.pairwise_synergy().flat, bins=10)
plt.show()

In [ ]:
# load p values estimated from empirical distributions
path = '/Users/zhangx/git/MIMIC_HPO/src/main/resources/p_value_map_584.obj'
with open(path, 'rb') as f:
    p_value_map = pickle.load(f)
p_renal_failure = p_value_map['584']
data = acute_renal_failure.pairwise_synergy_labeled_with_p_values(p_renal_failure)
data.reset_index(drop=True).head()

In [ ]:
mask = np.array([hpo.has_dependency(data.P1[i], data.P2[i]) for i in np.arange(data.shape[0])])
S_acute_renal_failure = data.loc[np.logical_not(mask), :].sort_values(by = 'synergy', ascending=False)
S_acute_renal_failure = S_acute_renal_failure.loc[S_acute_renal_failure.P1 < S_acute_renal_failure.P2, :]
S_acute_renal_failure['P1_label'] = np.array([hpo.term_id2name_map().get(termid) for termid in S_acute_renal_failure.P1])
S_acute_renal_failure['P2_label'] = np.array([hpo.term_id2name_map().get(termid) for termid in S_acute_renal_failure.P2])
top5percent_synergy_pair_acute_renal_failure = S_acute_renal_failure.iloc[0:math.ceil(0.05 * len(S_acute_renal_failure)), :]
top5percent_synergy_pair_acute_renal_failure.head(20)

In [ ]:
top5percent_synergy_pair_acute_renal_failure.to_csv('top5percent_synergy_pair_acute_renal_failure.csv', index=False)

In [ ]:
sepsis = deserialized['038']
plt.hist(sepsis.pairwise_synergy().flat, bins=10)
plt.show()

In [ ]:
# load p values estimated from empirical distributions
path = '/Users/zhangx/git/MIMIC_HPO/src/main/resources/p_value_map_038.obj'
with open(path, 'rb') as f:
    p_value_map = pickle.load(f)
p_sepsis = p_value_map['038']
data = sepsis.pairwise_synergy_labeled_with_p_values(p_sepsis)
data.reset_index(drop=True).head()

In [ ]:
mask = np.array([hpo.has_dependency(data.P1[i], data.P2[i]) for i in np.arange(data.shape[0])])
S_sepsis = data.loc[np.logical_not(mask), :].sort_values(by = 'synergy', ascending=False)
S_sepsis = S_sepsis.loc[S_sepsis.P1 < S_sepsis.P2, :]
top5percent = S_sepsis.shape[0] * 0.05
top5percent_synergy_pair_sepsis = S_sepsis.reset_index(drop=True).iloc[0:int(top5percent), :]
top5percent_synergy_pair_sepsis['P1_label'] = np.array([hpo.term_id2name_map().get(termid) for termid in top5percent_synergy_pair_sepsis.P1])
top5percent_synergy_pair_sepsis['P2_label'] = np.array([hpo.term_id2name_map().get(termid) for termid in top5percent_synergy_pair_sepsis.P2])



In [ ]:
top5percent_synergy_pair_sepsis.head(n = 100)

In [ ]:
top5percent_synergy_pair_sepsis.to_csv('top5percent_synergy_pair_sepsis.csv', index=False)

## Synergy between Radiology- and Lab-derived Abnormal Phenotypes

In [ ]:
with open('synergies_radiology_lab_primary_and_secondary.obj', 'rb') as synergies_file:
    synergies_rad_lab = pickle.load(synergies_file)

In [ ]:
len(synergies_rad_lab)

In [6]:
def load_p_values(path):
    with open(path, 'rb') as f:
        p = pickle.load(f)
    return p

def filtered_synergy_dataframe(synergy, p_values=None, percentile_cut=None):
    if p_values is not None:
        data = synergy.pairwise_synergy_labeled_with_p_values(p_values)
    else:
        data = synergy.pairwise_synergy_labeled()
    # remove directly dependent terms 
    mask = np.array([hpo.has_dependency(data.P1[i], data.P2[i]) for i in np.arange(data.shape[0])])
    data_filtered = data.loc[np.logical_not(mask), :].sort_values(by = 'synergy', ascending=False)
    data_filtered = data_filtered.loc[data_filtered.P1 < data_filtered.P2, :]
    data_filtered['P1_radiology_label'] = np.array([hpo.term_id2name_map().get(termid) for termid in data_filtered.P1])
    data_filtered['P2_lab_label'] = np.array([hpo.term_id2name_map().get(termid) for termid in data_filtered.P2])
    
    
    if (percentile_cut == None):
        percentile = 1
    else:
        percentile = percentile_cut / 100
    top_percentile = data_filtered.iloc[0:math.ceil(percentile * len(data_filtered)), :]
    
    
    I, II = synergy.mutual_information()
    Ia, Ib = I.values()
    p1_labels, p2_labels = synergy.vars_labels.values()
    mf_P1 = pd.DataFrame(data={'P1': p1_labels, 'mf_d_P1': Ia})
    mf_P2 = pd.DataFrame(data={'P2': p2_labels, 'mf_d_P2': Ib})
    mf_d_P1P2 = pd.DataFrame(data={'P1': np.repeat(p1_labels, len(p2_labels)), 'P2': np.tile(p2_labels, [len(p1_labels)]), 'mf_d_P1P2': II.flat})
    fully_labeled = top_percentile.merge(mf_P1, on='P1').merge(mf_P2, on='P2').merge(mf_d_P1P2, on=['P1', 'P2'])
    
    return fully_labeled

In [ ]:
#p_values = load_p_values('p_value_428.obj')
filtered_data = filtered_synergy_dataframe(synergies_rad_lab, icd = '428', icd_label = 'heart_failure', p_values=None, percentile_cut = 5)
filtered_data.to_csv('synergy-radiology_labtest-428_primary_and_secondary.csv', index=False)

In [ ]:
#p_value = load_p_values('p_value_584.obj')
filtered_data = filtered_synergy_dataframe(synergies_rad_lab, icd = '584', icd_label = 'acute_renal_failure', p_values=None, percentile_cut = 5)
filtered_data.to_csv('synergy-radiology-labtest-584_primary_and_secondary.csv', index=False)

In [ ]:
#p_values = load_p_values('p_value_038.obj')
filtered_data = filtered_synergy_dataframe(synergies_rad_lab, icd = '038', icd_label = 'sepsis', p_values=None, percentile_cut = 5)
filtered_data.to_csv('synergy-radiology_labtest-038_primary_and_secondary.csv', index=False)

### Just look at primary diagnosis


In [3]:
mf_all = pd.read_csv('mutual_info_textHpo_labHpo.csv')
mf_all.head()

,P1,P2,entropy_P1,entropy_P2,mf_P1_P2
0,HP:0000001,HP:0000118,0.367357,0.142722,0.011065
1,HP:0000001,HP:0000001,0.367357,0.142722,0.011065
2,HP:0000001,HP:0001939,0.367357,0.180924,0.014722
3,HP:0000001,HP:0001871,0.367357,0.206220,0.019528
4,HP:0000001,HP:0001877,0.367357,0.239249,0.017066


In [4]:
with open('synergies_radiology_lab_primary_only.obj', 'rb') as synergies_file:
    synergies_rad_lab_primary_only = pickle.load(synergies_file)

In [7]:
#p_values = load_p_values('p_value_428_primary_only.obj')
synergy = synergies_rad_lab_primary_only['428']
filtered_data = filtered_synergy_dataframe(synergy, p_values=None, percentile_cut = 5)
include_overall_mf = filtered_data.merge(mf_all, on=['P1', 'P2'], how='left')
include_overall_mf['entropy_P1_given_d'] = include_overall_mf['entropy_P1'] - include_overall_mf['mf_d_P1']
include_overall_mf['entropy_P2_given_d'] = include_overall_mf['entropy_P2'] - include_overall_mf['mf_d_P2']


In [8]:
include_overall_mf.head()

,P1,P2,synergy,P1_radiology_label,P2_lab_label,mf_d_P1,mf_d_P2,mf_d_P1P2,entropy_P1,entropy_P2,mf_P1_P2,entropy_P1_given_d,entropy_P2_given_d
0,HP:0002202,HP:0004363,0.002496,Pleural effusion,Abnormal circulating calcium concentration,0.002937,0.000047,0.005479,0.966136,0.869158,0.102352,0.963200,0.869111
1,HP:0001640,HP:0004363,0.001359,Cardiomegaly,Abnormal circulating calcium concentration,0.007596,0.000047,0.009002,0.819058,0.869158,0.036363,0.811462,0.869111
2,HP:0001635,HP:0004363,0.001114,Congestive heart failure,Abnormal circulating calcium concentration,0.011296,0.000047,0.012457,0.420952,0.869158,0.013604,0.409657,0.869111
3,HP:0000969,HP:0004363,0.000966,Edema,Abnormal circulating calcium concentration,0.001782,0.000047,0.002795,0.930309,0.869158,0.071260,0.928527,0.869111
4,HP:0002086,HP:0004363,0.000759,Abnormality of the respiratory system,Abnormal circulating calcium concentration,0.001930,0.000047,0.002736,0.671973,0.869158,0.080085,0.670043,0.869111


In [10]:
include_overall_mf.sort_values(by='synergy', ascending=False).head(20)
include_overall_mf.sort_values(by='synergy', ascending=False).to_csv('synergy-radiology_labtest_primary_only-428_corrected.csv', index=False)

In [16]:
#p_values = load_p_values('p_value_584_primary_only.obj')
synergy = synergies_rad_lab_primary_only['584']
filtered_data = filtered_synergy_dataframe(synergy, p_values=None, percentile_cut = 5)
include_overall_mf = filtered_data.merge(mf_all, on=['P1', 'P2'], how='left')
include_overall_mf['entropy_P1_given_d'] = include_overall_mf['entropy_P1'] - include_overall_mf['mf_d_P1']
include_overall_mf['entropy_P2_given_d'] = include_overall_mf['entropy_P2'] - include_overall_mf['mf_d_P2']

In [17]:
include_overall_mf.sort_values(by='synergy', ascending=False).head(20)
#include_overall_mf.sort_values(by='synergy', ascending=False).to_csv('synergy-radiology_labtest_primary_only-584_corrected.csv', index=False)

,P1,P2,synergy,P1_radiology_label,P2_lab_label,mf_d_P1,mf_d_P2,mf_d_P1P2,entropy_P1,entropy_P2,mf_P1_P2,entropy_P1_given_d,entropy_P2_given_d
0,HP:0000083,HP:0012419,0.000876,Renal insufficiency,Hyperoxemia,5.441291e-03,0.000861,0.007179,0.538881,0.961683,0.011766,0.533440,0.960822
474,HP:0002202,HP:0003259,0.000696,Pleural effusion,Elevated serum creatinine,2.903760e-07,0.015268,0.015964,0.966136,0.847731,0.026262,0.966136,0.832463
69,HP:0000083,HP:0032369,0.000677,Renal insufficiency,Alkalemia,5.441291e-03,0.000522,0.006640,0.538881,0.847660,0.011645,0.533440,0.847138
485,HP:0002202,HP:0002901,0.000650,Pleural effusion,Hypocalcemia,2.903760e-07,0.001232,0.001883,0.966136,0.898148,0.098159,0.966136,0.896916
135,HP:0000083,HP:0012101,0.000635,Renal insufficiency,Decreased serum creatinine,5.441291e-03,0.000583,0.006659,0.538881,0.899611,0.010167,0.533440,0.899028
495,HP:0002202,HP:0011422,0.000611,Pleural effusion,Abnormal blood chloride concentration,2.903760e-07,0.002015,0.002626,0.966136,0.962195,0.055719,0.966136,0.960180
2,HP:0001919,HP:0012419,0.000600,Acute kidney injury,Hyperoxemia,9.108212e-03,0.000861,0.010569,0.173706,0.961683,0.001628,0.164598,0.960822
507,HP:0002202,HP:0012100,0.000598,Pleural effusion,Abnormal circulating creatinine level,2.903760e-07,0.011753,0.012351,0.966136,0.987241,0.043258,0.966136,0.975488
136,HP:0002202,HP:0012101,0.000592,Pleural effusion,Decreased serum creatinine,2.903760e-07,0.000583,0.001175,0.966136,0.899611,0.034825,0.966136,0.899028
564,HP:0002107,HP:0020062,0.000569,Pneumothorax,Decreased hemoglobin concentration,8.144860e-04,0.001844,0.003227,0.659875,0.938389,0.046457,0.659061,0.936545


In [14]:
#p_values = load_p_values('p_value_038_primary_only.obj')
synergy = synergies_rad_lab_primary_only['038']
filtered_data = filtered_synergy_dataframe(synergy, p_values=None, percentile_cut = 5)
include_overall_mf = filtered_data.merge(mf_all, on=['P1', 'P2'], how='left')
include_overall_mf['entropy_P1_given_d'] = include_overall_mf['entropy_P1'] - include_overall_mf['mf_d_P1']
include_overall_mf['entropy_P2_given_d'] = include_overall_mf['entropy_P2'] - include_overall_mf['mf_d_P2']

In [15]:
include_overall_mf.sort_values(by='synergy', ascending=False).head(20)
include_overall_mf.sort_values(by='synergy', ascending=False).to_csv('synergy-radiology_labtest_primary_only-038_corrected.csv', index=False)